In [2]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [3]:
import splitfolders

data_d = '/kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)'
output_d = '/kaggle/working/splitted_Data'
splitfolders.ratio(data_d, output=output_d, seed=1942, ratio=(.8, .15, .05))

Copying files: 41793 files [07:19, 95.00 files/s] 


In [5]:
train = "/kaggle/working/splitted_Data/train"
test = "/kaggle/working/splitted_Data/test"
validation = "/kaggle/working/splitted_Data/val"

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
 
input_shape = (224, 224, 1)
 
base_dnn_model = Sequential()
# Flatten the input image to 1D
base_dnn_model.add(Flatten(input_shape=input_shape))
   
 # First dense layer
base_dnn_model.add(Dense(128, activation='relu'))
   
# Second dense layer
base_dnn_model.add(Dense(64, activation='relu'))
 
# Third hidden layer
base_dnn_model.add(Dense(32, activation='relu'))
   
# Output layer for binary classification
base_dnn_model.add(Dense(1, activation='sigmoid'))
   
# Compile the model
base_dnn_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
 
base_dnn_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_5 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │     6,422,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,433,025 (24.54 MB)

 Trainable params: 6,433,025 (24.54 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Preprocessing function
def preprocess_function(img):
    img_array = img.astype('uint8')
    img_array = cv2.GaussianBlur(img_array, (5, 5), 0)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=-1)
    return img_array

# Initiate data generators with the custom preprocessing function
train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_function, 
                                    rotation_range=10,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.1)
val_data_gen = ImageDataGenerator(preprocessing_function=preprocess_function)
test_data_gen = ImageDataGenerator(preprocessing_function=preprocess_function)

# Create the generators
train_generator = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    color_mode='grayscale', 
    class_mode='binary'  
)

val_generator = val_data_gen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary',
    shuffle = False
)

test_generator = test_data_gen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary',
    shuffle=False
)

# Add EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
base_history = base_dnn_model.fit(train_generator, validation_data=val_generator, epochs=50, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = base_dnn_model.evaluate(test_generator)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

Found 33434 images belonging to 2 classes.
Found 6268 images belonging to 2 classes.
Found 2091 images belonging to 2 classes.
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1045/1045 ━━━━━━━━━━━━━━━━━━━━ 260s 246ms/step - accuracy: 0.6224 - loss: 1.0164 - val_accuracy: 0.8789 - val_loss: 0.2951
Epoch 2/50
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 259s 246ms/step - accuracy: 0.8475 - loss: 0.3514 - val_accuracy: 0.9813 - val_loss: 0.0798
Epoch 3/50
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 257s 244ms/step - accuracy: 0.9112 - loss: 0.2257 - val_accuracy: 0.9697 - val_loss: 0.0977
Epoch 4/50
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 258s 245ms/step - accuracy: 0.9213 - loss: 0.1989 - val_accuracy: 0.9767 - val_loss: 0.0789
Epoch 5/50
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 257s 244ms/step - accuracy: 0.9431 - loss: 0.1519 - val_accuracy: 0.9813 - val_loss: 0.0620
Epoch 6/50
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 257s 245ms/step - accuracy: 0.9500 - loss: 0.1434 - val_accuracy: 0.9925 - val_loss: 0.0263
Epoch 7/50
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 256s 244ms/step - accuracy: 0.9654 - loss: 0.1006 - val_accuracy: 0.9799 - val_loss: 0.0564
Epoch 8/50
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 255s 243ms/step - accuracy: 0.9

In [18]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
# Initialize the data generator for validation data
valid_datagen = ImageDataGenerator(rescale=1 / 255.0)
 
valid_generator = valid_datagen.flow_from_directory(
    directory="/kaggle/working/splitted_Data/val",
    target_size=(224, 224),
    color_mode="grayscale",
    class_mode="binary",
    batch_size=32,
    shuffle=False
)
 
steps_per_epoch = np.ceil(valid_generator.samples / valid_generator.batch_size).astype(int)
 
# Get predictions from the model
predictions = base_dnn_model.predict(valid_generator, steps=steps_per_epoch)
 
# Since this is a binary classification, we threshold predictions at 0.5
predicted_classes = (predictions > 0.5).astype(int).reshape(-1)
 
# Get the true classes
true_classes = valid_generator.classes
 
# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))
 
# Print the classification report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=list(valid_generator.class_indices.keys())))

Found 6268 images belonging to 2 classes.
196/196 ━━━━━━━━━━━━━━━━━━━━ 17s 84ms/step
Confusion Matrix:
[[3339   13]
 [  33 2883]]

Classification Report:
              precision    recall  f1-score   support

      Drowsy       0.99      1.00      0.99      3352
  Non Drowsy       1.00      0.99      0.99      2916

    accuracy                           0.99      6268
   macro avg       0.99      0.99      0.99      6268
weighted avg       0.99      0.99      0.99      6268

